# Training Model
1. Keras on Tensorflow
2. hyper-parameter search
3. vanila-Unet: pull the hyper parameter setting out from the function of model
4. date: 01312020


In [1]:
import os, sys
import cv2
import numpy as np
import uuid
from skimage.io import imread, imsave, imshow
from PIL import Image, ImageTk
import matplotlib.pyplot as plt
from imutils import paths
import itertools
import json
from pprint import pprint



import tensorflow as tf
from tensorflow.keras import backend as K

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from core.imageprep import dir_checker, random_crop, crop_generator, random_crop_batch
from core.models import UNet, UNet_hp, vanilla_unet, vanilla_unet_nodrop
from core.metrics import iou_coef, dice_coef

from tensorflow.python.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, ReduceLROnPlateau
from tensorboard.plugins.hparams import api as hp
from datetime import datetime

from IPython import get_ipython
# %load_ext autoreload
get_ipython().run_line_magic('load_ext', 'autoreload')
# %autoreload 2
get_ipython().run_line_magic('autoreload', '2')
# %load_ext tensorboard
get_ipython().run_line_magic('load_ext', 'tensorboard')

In [2]:
from packaging import version
print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.0.0


In [3]:
sys.executable

'C:\\Users\\wucci_admin\\Anaconda3\\envs\\tfdl02\\python.exe'

 ## Load Training Dataset

In [4]:
# load image
print("Load Images...")
# on mac
# path = "/Volumes/LaCie_DataStorage/PerlmutterData/"

# on Window PC 
path = os.path.join('D:', 'PerlmutterData')

# input set
crop_input_set = '2020_01_23_09_51_20x'
# crop_input_set = '2020_01_30_11_24_1x' # small training set

imginput = os.path.join('dl_seg_project_raw', 'data_crop', crop_input_set,)
imgpath = os.path.join(path, imginput)

print(imgpath)

img_dir = os.path.join(imgpath, 'images')
label_dir = os.path.join(imgpath, 'labels')
print(img_dir)
print(label_dir)

# check if the output folder exist. If not, create a folder
dir_checker('logs', path)
path_logs = os.path.join(path, 'logs')
dir_checker('fit', path_logs)
dir_checker('model', path_logs)
dir_checker('pars', path_logs)

Load Images...
D:PerlmutterData\dl_seg_project_raw\data_crop\2020_01_23_09_51_20x
D:PerlmutterData\dl_seg_project_raw\data_crop\2020_01_23_09_51_20x\images
D:PerlmutterData\dl_seg_project_raw\data_crop\2020_01_23_09_51_20x\labels
logs exists in D:PerlmutterData
fit exists in D:PerlmutterData\logs
model exists in D:PerlmutterData\logs
pars exists in D:PerlmutterData\logs


 Print the first file name.

In [5]:
imgpath_all = list(paths.list_images(imgpath))
print(imgpath_all[0])

D:PerlmutterData\dl_seg_project_raw\data_crop\2020_01_23_09_51_20x\images\nucleus\0001.tif


 ## Create Image Datagenerator
 1. create only one datagen
 2. specify valiation split in datagen argument
 3. add split data when calling `datagen.flow_from_directory`

In [6]:
timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M")
date =  datetime.now().strftime("%Y_%m_%d")
seed = 103
batch_size = 16
epoch = 20
validation_steps = 20
validation_split = 0.1
training_sample_size = len(imgpath_all)
IMG_HEIGHT = None
IMG_WIDTH = None
classes = ['cell_membrane', 'nucleus', 'autophagosome']
inputclass = [classes[1]]
learning_rate = 1e-4
loss = "binary_crossentropy"
metrics = ['accuracy', iou_coef, dice_coef]

metrics_name = []
for f in metrics:
    if callable(f):
        metrics_name.append(f.__name__)
    else:
        metrics_name.append(f)

In [7]:
# create arguments for data generator
data_gen_img_args = dict(
                # featurewise_center = True,
                # featurewise_std_normalization = True,
                horizontal_flip = True,
                vertical_flip = True,
                rotation_range = 90.,
                width_shift_range = 0.1,
                height_shift_range = 0.1,
                shear_range = 0.07,
                zoom_range = 0.2,
                validation_split = validation_split, # <- specify validation_split ratio
                # fill_mode='constant',
                # cval=0.,
                rescale=1.0/255.0,
                )

data_gen_label_args = dict(
                # featurewise_center=True,
                # featurewise_std_normalization=True,
                horizontal_flip = True,
                vertical_flip = True,
                rotation_range = 90.,
                width_shift_range = 0.1,
                height_shift_range = 0.1,
                shear_range = 0.07,
                zoom_range = 0.2,
                validation_split = validation_split, # <- specify validation_split ratio
                # fill_mode='constant',
                # cval=0.,
                # rescale=1.0/255.0,
                rescale=1.0/255.0,
                )

# create parameter
pars = dict(
                # basic information
                timestamp = timestamp,
                date = date,
                seed = seed,
                batch_size = batch_size,
                
                # Data generator
                crop_input_set = crop_input_set,
                validation_steps = validation_steps,
                validation_split = validation_split,
                training_sample_size = training_sample_size,
                
                # training class
                classes = classes,
                inputclass = inputclass,
    
                # add datagen args
                data_gen_img_args = data_gen_img_args,
                data_gen_label_args = data_gen_label_args,
                
                # Build model
                IMG_HEIGHT = IMG_HEIGHT,
                IMG_WIDTH = IMG_WIDTH,
                epoch = epoch, 
                loss = loss,
                metrics_name = metrics_name,
                learning_rate = learning_rate,
                )

# save parameter
path_pars = os.path.join(path_logs, 'pars')
dir_checker(inputclass[0], path_pars)
dir_checker(date, os.path.join(path_pars, inputclass[0]))

pprint(pars)

nucleus exists in D:PerlmutterData\logs\pars
2020_02_05 exists in D:PerlmutterData\logs\pars\nucleus
{'IMG_HEIGHT': None,
 'IMG_WIDTH': None,
 'batch_size': 16,
 'classes': ['cell_membrane', 'nucleus', 'autophagosome'],
 'crop_input_set': '2020_01_23_09_51_20x',
 'data_gen_img_args': {'height_shift_range': 0.1,
                       'horizontal_flip': True,
                       'rescale': 0.00392156862745098,
                       'rotation_range': 90.0,
                       'shear_range': 0.07,
                       'validation_split': 0.1,
                       'vertical_flip': True,
                       'width_shift_range': 0.1,
                       'zoom_range': 0.2},
 'data_gen_label_args': {'height_shift_range': 0.1,
                         'horizontal_flip': True,
                         'rescale': 0.00392156862745098,
                         'rotation_range': 90.0,
                         'shear_range': 0.07,
                         'validation_split': 0.1,
   

In [8]:
par_file_dir = os.path.join(path_pars, inputclass[0], date, 'pars_' + timestamp + '.json')
print(par_file_dir)

with open(par_file_dir, 'w') as outfile:
    json.dump(pars, outfile, indent=4)


D:PerlmutterData\logs\pars\nucleus\2020_02_05\pars_2020_02_05_16_29.json


In [9]:
# create generator
image_datagen = ImageDataGenerator(**data_gen_img_args)
label_datagen = ImageDataGenerator(**data_gen_label_args)

In [10]:
# load images into generator
train_image_generator = image_datagen.flow_from_directory(
    img_dir,
    class_mode=None,
    classes=inputclass,
    color_mode='grayscale',
    batch_size=batch_size,
    subset='training', # <- define subset as 'training'
    seed=seed)

train_label_generator = label_datagen.flow_from_directory(
    label_dir,
    class_mode=None,
    classes=inputclass,
    color_mode='grayscale',
    batch_size=batch_size,
    subset='training',
    seed=seed)

valid_image_generator = image_datagen.flow_from_directory(
    img_dir,
    class_mode=None,
    classes=inputclass,
    color_mode='grayscale',
    batch_size=batch_size,
    subset='validation', # <- define subset as 'validation'
    seed=seed)

valid_label_generator = label_datagen.flow_from_directory(
    label_dir,
    class_mode=None,
    classes=inputclass,
    color_mode='grayscale',
    batch_size=batch_size,
    subset='validation',
    seed=seed)

Found 5958 images belonging to 1 classes.
Found 5958 images belonging to 1 classes.
Found 662 images belonging to 1 classes.
Found 662 images belonging to 1 classes.


In [11]:

# merge image and label generator
def combine_generator(gen1, gen2):
    while True:
        yield(gen1.next(), gen2.next()) 
train_generator = combine_generator(train_image_generator, train_label_generator)
valid_generator = combine_generator(valid_image_generator, valid_label_generator)


In [12]:
'''
train_generator = zip(train_image_generator, train_label_generator)
valid_generator = zip(valid_image_generator, valid_label_generator)
'''

'\ntrain_generator = zip(train_image_generator, train_label_generator)\nvalid_generator = zip(valid_image_generator, valid_label_generator)\n'

 ## Training

In [13]:
print("Start training...")

Start training...


 ### Define Callbacks

 ### Setup the model

In [14]:
# calculate steps_per_epoch
steps_per_epoch = training_sample_size * (1 - validation_split) // batch_size
print("Steps per epoch: {}".format(steps_per_epoch))

Steps per epoch: 744.0


## Hyper-parameters grid search

In [15]:
# Hyperparameter
# Create a .v2 file for saving hyperparameter and evaluation
# so we can see the results on tensorboard
hparamtuning_dir = os.path.join(path_logs, 'fit', inputclass[0], date, timestamp)

HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.7, 0.5]))
HP_LAYERS = hp.HParam('layers', hp.Discrete([5, 4]))

# hparams_list = [HP_DROPOUT, HP_LAYERS]

with tf.summary.create_file_writer(hparamtuning_dir).as_default():
    hp.hparams_config(
        hparams=[HP_DROPOUT, HP_LAYERS],
        metrics=[hp.Metric('accuracy', display_name='Accuracy'), 
                 hp.Metric('iou_coef', display_name='IoU_Coef'), # create container for customized metrics
                 hp.Metric('dice_coef', display_name='Dice_Coef')], # the same
  )


In [16]:
def run(run_name, hparamtuning_dir, hparams):
    
    # checkpoint
    modelfilename = 'model_' + timestamp + '.h5'
    dir_checker(run_name, hparamtuning_dir)
    dir_checker('model', os.path.join(hparamtuning_dir, run_name))
    
    modelfile_path = os.path.join(hparamtuning_dir, run_name, 'model', modelfilename)
    checkpointer = ModelCheckpoint(filepath = modelfile_path, 
                                   monitor = 'val_accuracy', 
                                   mode = 'max', 
                                   verbose = 1, 
                                   save_best_only = True, 
                                  )

    # early stopping 
    early_stopping = EarlyStopping(monitor='val_loss',
                               patience=8,
                               verbose=1,
                               min_delta=1e-4)

    # learning rate adjustment
    reduceLR = ReduceLROnPlateau(monitor='val_loss',
                        factor=0.1,
                        patience=4,
                        verbose=1,
                        min_delta=1e-4)

    # tensorboard ----------------------------------------------
    
    # file_writer = create_file_writer(os.path.join(path_logs, 'fit', inputclass[0], date, timestamp, "metrics"))
    # file_writer.set_as_default()

    metrics = ['accuracy', iou_coef, dice_coef]
    
    tensorboard_callback = TensorBoard(log_dir = os.path.join(hparamtuning_dir, run_name), 
                                       profile_batch = 0, 
                                       update_freq= 30,
                                       histogram_freq = 1
                                       )

    # compile callbacks
    # callbacks = [checkpointer, tensorboard_callback, early_stopping, reduceLR]
    callbacks = [checkpointer, reduceLR, tensorboard_callback]
    
    hparamtuning_runname_dir = os.path.join(hparamtuning_dir, run_name)
    
    
    with tf.summary.create_file_writer(hparamtuning_runname_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial

        # prepare the model -----------------------------------
        
        # load hyper-parameter
        dropout = float(hparams[HP_DROPOUT])
        print('dropout: {}'.format(dropout))
        
        num_layers = int(hparams[HP_LAYERS])
        print('num layers: {}'.format(num_layers))
        
        unetmodel = vanilla_unet_nodrop(
                            shape = (IMG_HEIGHT, IMG_WIDTH), 
                            dropout = dropout, 
                            num_layers = num_layers, 
                            lr = learning_rate, 
                            loss = loss,
                            metrics = metrics,
                            summary = False,
                           )
        
        # load model ------------------------------------------
        
        '''
        # load weight
        path_model = os.path.join('D:', 'PerlmutterData', 'logs', 'model', 
                                'nucleus', 
                                '2019_12_13',
                                '2019_12_13_17_28',
                                'run-1', 
                                'model_2019_12_13_17_28.h5',)
        
        unetmodel.load_weights(path_model)
        '''
        
        
        # train the model -------------------------------------
        unetmodel.fit_generator(
                            generator = train_generator, 
                            validation_data = valid_generator,
                            validation_steps = validation_steps,
                            steps_per_epoch = steps_per_epoch,
                            epochs = epoch,  
                            callbacks = callbacks,
                            verbose = 1, 
                            )
    
        _, accuracy, iou, dice,  = unetmodel.evaluate_generator(valid_generator, steps = 50, verbose=1)
        tf.summary.scalar('accuracy', accuracy, step = 1)
        tf.summary.scalar('iou_coef', iou, step = 1)
        tf.summary.scalar('dice_coef', dice, step = 1)
        
        # -----------------------------------------------------
        
        # clean memory
        K.clear_session()
        del unetmodel

In [ ]:
#train the model
session_num = 0

for dropout in HP_DROPOUT.domain.values:
    for layer in HP_LAYERS.domain.values:
        
        run_name = "run-{}".format(session_num)
        print('--- Starting trial: {}'.format(run_name))
        
        # create hyper-parameter
        hparams = {
            HP_DROPOUT: dropout,
            HP_LAYERS: layer,
        }
        print(hparams)
        print({h.name: hparams[h] for h in hparams})

        # build model and traning
        run(run_name, hparamtuning_dir, hparams)
        
        session_num += 1


--- Starting trial: run-0
{HParam(name='dropout', domain=Discrete([0.5, 0.7]), display_name=None, description=None): 0.5, HParam(name='layers', domain=Discrete([4, 5]), display_name=None, description=None): 4}
{'dropout': 0.5, 'layers': 4}
run-0 does not exist in D:PerlmutterData\logs\fit\nucleus\2020_02_05\2020_02_05_16_29
model does not exist in D:PerlmutterData\logs\fit\nucleus\2020_02_05\2020_02_05_16_29\run-0
dropout: 0.5
num layers: 4
Epoch 1/20
743/744 [============================>.] - ETA: 0s - loss: 0.4338 - accuracy: 0.7971 - iou_coef: 0.1975 - dice_coef: 0.2548
Epoch 00001: val_accuracy improved from -inf to 0.63900, saving model to D:PerlmutterData\logs\fit\nucleus\2020_02_05\2020_02_05_16_29\run-0\model\model_2020_02_05_16_29.h5
744/744 [==============================] - 647s 869ms/step - loss: 0.4339 - accuracy: 0.7970 - iou_coef: 0.1975 - dice_coef: 0.2548 - val_loss: 0.5973 - val_accuracy: 0.6390 - val_iou_coef: 0.2157 - val_dice_coef: 0.3075
Epoch 2/20
743/744 [======

Epoch 18/20
743/744 [============================>.] - ETA: 0s - loss: 0.0966 - accuracy: 0.9673 - iou_coef: 0.3809 - dice_coef: 0.4009
Epoch 00018: val_accuracy did not improve from 0.97677

Epoch 00018: ReduceLROnPlateau reducing learning rate to 9.999999974752428e-08.
744/744 [==============================] - 635s 853ms/step - loss: 0.0965 - accuracy: 0.9674 - iou_coef: 0.3808 - dice_coef: 0.4008 - val_loss: 0.2332 - val_accuracy: 0.9209 - val_iou_coef: 0.4462 - val_dice_coef: 0.4870
Epoch 19/20
743/744 [============================>.] - ETA: 0s - loss: 0.0981 - accuracy: 0.9658 - iou_coef: 0.3817 - dice_coef: 0.4019
Epoch 00019: val_accuracy did not improve from 0.97677
744/744 [==============================] - 634s 853ms/step - loss: 0.0980 - accuracy: 0.9658 - iou_coef: 0.3820 - dice_coef: 0.4022 - val_loss: 0.2513 - val_accuracy: 0.9143 - val_iou_coef: 0.4730 - val_dice_coef: 0.5198
Epoch 20/20
743/744 [============================>.] - ETA: 0s - loss: 0.0956 - accuracy: 0.967

Epoch 15/20
743/744 [============================>.] - ETA: 1s - loss: 0.1005 - accuracy: 0.9684 - iou_coef: 0.3865 - dice_coef: 0.4075
Epoch 00015: val_accuracy did not improve from 0.92257

Epoch 00015: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-07.
744/744 [==============================] - 826s 1s/step - loss: 0.1004 - accuracy: 0.9685 - iou_coef: 0.3867 - dice_coef: 0.4077 - val_loss: 0.4771 - val_accuracy: 0.8476 - val_iou_coef: 0.3836 - val_dice_coef: 0.4140
Epoch 16/20
743/744 [============================>.] - ETA: 1s - loss: 0.0940 - accuracy: 0.9683 - iou_coef: 0.3880 - dice_coef: 0.4081
Epoch 00016: val_accuracy did not improve from 0.92257
744/744 [==============================] - 826s 1s/step - loss: 0.0940 - accuracy: 0.9683 - iou_coef: 0.3882 - dice_coef: 0.4083 - val_loss: 0.6490 - val_accuracy: 0.8066 - val_iou_coef: 0.3210 - val_dice_coef: 0.3443
Epoch 17/20
743/744 [============================>.] - ETA: 1s - loss: 0.0928 - accuracy: 0.9683 - io

Epoch 9/20
743/744 [============================>.] - ETA: 0s - loss: 0.1439 - accuracy: 0.9596 - iou_coef: 0.3543 - dice_coef: 0.3802
Epoch 00009: val_accuracy did not improve from 0.85639
744/744 [==============================] - 635s 853ms/step - loss: 0.1438 - accuracy: 0.9596 - iou_coef: 0.3540 - dice_coef: 0.3799 - val_loss: 0.7439 - val_accuracy: 0.7886 - val_iou_coef: 0.3091 - val_dice_coef: 0.3459
Epoch 10/20
743/744 [============================>.] - ETA: 0s - loss: 0.1377 - accuracy: 0.9612 - iou_coef: 0.3610 - dice_coef: 0.3864
Epoch 00010: val_accuracy did not improve from 0.85639
744/744 [==============================] - 634s 852ms/step - loss: 0.1379 - accuracy: 0.9611 - iou_coef: 0.3608 - dice_coef: 0.3862 - val_loss: 0.5352 - val_accuracy: 0.8203 - val_iou_coef: 0.3514 - val_dice_coef: 0.3912
Epoch 11/20
743/744 [============================>.] - ETA: 0s - loss: 0.1399 - accuracy: 0.9606 - iou_coef: 0.3586 - dice_coef: 0.3845
Epoch 00011: val_accuracy improved from 0

744/744 [==============================] - 829s 1s/step - loss: 0.4475 - accuracy: 0.7768 - iou_coef: 0.1838 - dice_coef: 0.2410 - val_loss: 0.8195 - val_accuracy: 0.7692 - val_iou_coef: 0.2581 - val_dice_coef: 0.2844
Epoch 6/20
743/744 [============================>.] - ETA: 1s - loss: 0.1901 - accuracy: 0.9416 - iou_coef: 0.3163 - dice_coef: 0.3466
Epoch 00006: val_accuracy improved from 0.76918 to 0.80408, saving model to D:PerlmutterData\logs\fit\nucleus\2020_02_05\2020_02_05_16_29\run-3\model\model_2020_02_05_16_29.h5
744/744 [==============================] - 831s 1s/step - loss: 0.1899 - accuracy: 0.9417 - iou_coef: 0.3165 - dice_coef: 0.3468 - val_loss: 0.7497 - val_accuracy: 0.8041 - val_iou_coef: 0.2840 - val_dice_coef: 0.3082
Epoch 7/20
743/744 [============================>.] - ETA: 1s - loss: 0.1571 - accuracy: 0.9557 - iou_coef: 0.3402 - dice_coef: 0.3666
Epoch 00007: val_accuracy did not improve from 0.80408
744/744 [==============================] - 825s 1s/step - loss:

In [ ]:
print("Training finished")